# Digital diktanalyse: Mønstre i mengden

_18.februar 2026_

<!-- Ankomst: introduksjon -->
Velkommen til kodeverksted på Nasjonalbiblioteket! 

<!--Presentasjon: hvem er jeg? Hva er konteksten? -->

<!--
Jeg heter Ingerid Løyning Dale, og jobber som språkteknolog i Språkbanken og DH-laben her på Nasjonalbiblioteket. 
Kodebiblioteket vi kommer til å gå gjennom i dag heter `poetry_analysis` og ble utviklet i samarbeid med Ranveig Kvinnsland fra Ibsensenteret på UiO, i et forskningsprosjekt som heter NORN. Det er et EU-finansiert prosjekt som undersøker nasjonalromantikken i litteraturhistorien. 

Vi har sett mest på dikt fra 1890-tallet, altså den såkalt nyromantiske perioden, da vi ikke skrev på nynorsk og bokmål, men landsmål og riksmål, som fortsatt var veldig likt dansk. 
Men det betyr ikke at dere er låst til 1800-tallsdiktning! Koden fungerer på all tekst der linjeskift er brukt for å skille mellom elementene som skal sammenlignes. Vi deler altså ikke teksten inn i setninger, men i linjer, og ser på gjentakelser på tvers av disse linjene. 
-->

<!-- Agenda -->
## Agenda
I løpet av dette verkstedet skal vi annotere ulike gjentakelsesmønstre i digitale tekster med regelbaserte algoritmer.

Vi skal gå gjennom tre forskjellige lyriske virkemidler som baserer seg på gjentakelser: 
1. [Enderim](#enderim)
2. [Bokstavrim](#bokstavrim) 
3. [Anafor](#anafor)

Med egne funksjoner for hvert av trekkene kan du blant annet undersøke om det ofte er de samme ordene som rimer på hverandre i enderimsmønstre, om bokstavrim dannes med vokaler eller konsonanter,  og hvilke ord som oftest brukes anaforisk.

<!-- Presentasjon: formål med verktøyet vi bruker, vise frem appen, forklare hvilke lyriske trekk vi ser på og hvordan disse defineres -->

## App 
I tillegg til kodebiblioteket, som kan brukes av alle med en pythonversjon høyere enn 3.11, har vi  utviklet en egen app der man kan eksperimentere med enkelttekster og visualisere annotasjonene av gjentakelsesmønstrene: 

https://dh.nb.no/run/diktanalyse/app/

<!--
Som eksempeldikt til demoen har jeg valgt ut et dikt som passer måneden vi er inne i:  
-->
### Eksempeldikt
> "Februarvise" fra *Høstblomster eller rimstubber* (1896), av Andreas Aabel.

In [10]:
from pathlib import Path

filepath = Path("eksempeldikt/Februarvise.txt")
text = filepath.read_text()

print(text)

Jeg elsker Februar. 
Om hun kan være knipsk og morsk, 
hun er saa egte norsk. 
aar snøen lyser borti li – 
dikkom, dakkom, dikkom, dikkom dakk – 
da ut paa kjælke, ut paa ski! 
Heiopsan, her er vi! 


Hun klæder sig med glans. 
Ja hun er vinterns bedste pryd 
og allerstørste fryd. 
Kun tumler sig paa fjord og fjell – 
dikkom, dakkom, dikkom, dikkom dakk! – 
i rennefok og rykemjell; 
men det er moro lell! 


Av alle systre tolv 
er dig det mindste livsløp undt, 
og rapp din dag gaar rundt. 
men ei vi sakner Solens guld – 
dikkom, dakkom, dikkom, dikkom dakk! – 
naar bare Maanen er os huld, 
helst om han er – lidt fuld. 


Og i din klare natt, 
naar ingen stor, men mange smaa 
sig tér paa himlens blaa, 
da tindrer vel med deiligt skjær – 
dikkom, dakkom, dikkom, dikkom dakk! – 
din endeløse stjernehær 
og tykkes mig saa nær. 


Min egen Februar! 
Lad andre vælge sig April 
med dens troløse smil! 
Jeg vælger dig, og til din pris – 
dikkom, dakkom, dikkom, dikkom dakk! – 
jeg synger, mens 

## Enderim

<!-- Enderim: 
- Operasjonalisering
- fallgruver
- hva er et nødrim?
- Er det rim når samme ordet gjentas?
- Utfordringer: lydlikhet vs. tekstlikhet, sammensatte ord 
- Hvilke ord rimer oftest?  
-->

Enderim definerer vi som stavelsesrim på slutten av en verselinje, altså at siste stavelse på én verselinje rimer på siste stavelse i en annen verselinje. 

Les mer om rim i *Store norske leksikon:* https://snl.no/rim 

<!--
I praksis er koden implementert slik:
- Vi går gjennom alle verselinjene i en strofe eller et dikt 
- For hver linje starter vi på bakerste bokstav, og sammenligner bokstav for bokstav, med siste ordet på alle tidligere linjer i strofen
- Dersom det er overlapp mellom to ord, og den delen som overlapper inneholder en vokal, så sier vi at ordene rimer.
- Da annoterer vi disse linjene med en merkelapp fra alfabetet.
- Sånn får vi mønstre som ABBA, AABB og ABAB i strofer på fire linjer der to og to av linjene rimer. 
-->


In [36]:
from poetry_analysis.rhyme_detection import tag_poem_file # type: ignore

# Annoter diktet i tekstfilen med funksjonen "tag_poem_file"
end_rhymes = tag_poem_file(filepath, write_to_file=True)


Resultatet er en liste med annotasjoner for hver strofe, som igjen inneholder en liste med annotasjoner per linje. 

Ved å flate ut den hierarkiske datastrukturen kan vi  laste inn annotasjonene i en dataramme og se nærmere på dem: 

In [37]:
import pandas as pd

# Flat ut listen med annotasjoner og last inn i en dataramme
flattened_data = []
for stanza in end_rhymes:
    stanza_id = stanza['stanza_id']
    rhyme_scheme = stanza['rhyme_scheme']
    for verse in stanza['verses']:
        flat_dict = verse.copy()
        # Disse to linjene bare rydder opp litt og fjerner noen felter vi ikke trenger
        del flat_dict["transcription"]
        del flat_dict["syllables"]
        # Legg til informasjon om strofen på hver av verselinjene
        flat_dict.update({
            'stanza_id': stanza_id,
            'rhyme_scheme': rhyme_scheme,
        })
        flattened_data.append(flat_dict)

# Lag en pandas dataramme
rhyme_df = pd.DataFrame(flattened_data)
rhyme_df

,rhyme_score,rhyme_tag,text,tokens,last_token,rhymes_with,verse_id,stanza_id,rhyme_scheme
0,0.0,a,Jeg elsker Februar.,"[jeg, elsker, februar]",februar,NaN,0,0,abbcdcc
1,0.0,b,"Om hun kan være knipsk og morsk,","[om, hun, kan, være, knipsk, og, morsk]",morsk,NaN,1,0,abbcdcc
2,1.0,b,hun er saa egte norsk.,"[hun, er, saa, egte, norsk]",norsk,1.0,2,0,abbcdcc
3,0.0,c,aar snøen lyser borti li –,"[aar, snøen, lyser, borti, li]",li,NaN,3,0,abbcdcc
4,0.0,d,"dikkom, dakkom, dikkom, dikkom dakk –","[dikkom, dakkom, dikkom, dikkom, dakk]",dakk,NaN,4,0,abbcdcc
5,1.0,c,"da ut paa kjælke, ut paa ski!","[da, ut, paa, kjælke, ut, paa, ski]",ski,3.0,5,0,abbcdcc
6,1.0,c,"Heiopsan, her er vi!","[heiopsan, her, er, vi]",vi,5.0,6,0,abbcdcc
7,0.0,a,Hun klæder sig med glans.,"[hun, klæder, sig, med, glans]",glans,NaN,0,1,abbcdcc
8,0.0,b,Ja hun er vinterns bedste pryd,"[ja, hun, er, vinterns, bedste, pryd]",pryd,NaN,1,1,abbcdcc
9,1.0,b,og allerstørste fryd.,"[og, allerstørste, fryd]",fryd,1.0,2,1,abbcdcc


### Rimordpar

Finnes det noen rimord som forekommer oftere enn andre i eksempeldiktet vårt? 

In [38]:
def find_rhyming_word(row: pd.Series, df: pd.DataFrame) -> str | None:
    rhyming_word = df["last_token"].to_numpy()[
        (df["stanza_id"] == row["stanza_id"])
        & (df["verse_id"] == row["rhymes_with"])
    ]
    return rhyming_word.item() if rhyming_word.any() else None


def add_rhyming_words(df: pd.DataFrame) -> pd.DataFrame:
    """Legg til kolonnen "rhyming_words" i datarammen `df`."""
    # Gjør om datatypen til verselinjenummeret så den er sammenlignbar med "rhymes_with_syll"
    df.loc[:, "verse_id"] = df["verse_id"].astype(float)
    df["rhyming_word"] = df.apply(
        lambda row: find_rhyming_word(row, df), axis=1
    )
    return df

def find_frequent_rhyme_pairs(df: pd.DataFrame) -> pd.DataFrame:
    df = add_rhyming_words(df)

    rhyming_pairs = df[
        [
            "verse_id",
            "rhymes_with",
            "rhyming_word",
            "last_token",
        ]
    ][df["rhyming_word"].notna()]
    #rhyming_pairs.to_csv("utdata/rimord_eksempeldikt.csv")
    rhyme_groups = (
        rhyming_pairs.groupby("rhyming_word")["last_token"]
        .value_counts()
        .sort_values(ascending=False)
    )
    return rhyme_groups


rhyme_groups = find_frequent_rhyme_pairs(rhyme_df)
rhyme_groups.to_csv("utdata/rimord_frekvens.csv")

rhyme_groups

rhyming_word  last_token
april         smil          1
fjell         rykemjell     1
guld          huld          1
huld          fuld          1
li            ski           1
morsk         norsk         1
pris          vis           1
pryd          fryd          1
rykemjell     lell          1
ski           vi            1
skjær         stjernehær    1
smaa          blaa          1
stjernehær    nær           1
undt          rundt         1
vis           is            1
Name: count, dtype: int64

## Bokstavrim

<!-- Bokstavrim: konsonantisk og vokalisk (men ikke assonans!)  -->
Bokstavrim, eller alliterasjon, definerer vi som gjentakelsen av en bokstav eller en språklyd i begynnelsen av påfølgende ord. 

Med `poetry_analysis` kan vi skille mellom bokstavrim som begynner på samme konsonant eller vokal. Vi har ikke implementert kode for å fange opp såkalt "vokalisk alliterasjon" eller det som kalles assonans, eller halvrim.

Les mer om alliterasjon og assonans: 
- https://snl.no/allitterasjon
- https://snl.no/assonans

### Finn alle bokstavrim

In [20]:
from poetry_analysis.alliteration import find_line_alliterations

# Vi gjenbruker eksempeldiktet vi lastet inn tidligere i variabelen "text"
alliterations = find_line_alliterations(text, letter_type="both")

alliterations

[['fjord', 'fjell'],
 ['heiopsan', 'her'],
 ['huld', 'helst'],
 ['vante', 'vis'],
 ['men', 'mange'],
 ['sakner', 'solens'],
 ['smaa', 'sig'],
 ['natt', 'naar'],
 ['av', 'alle'],
 ['dikkom', 'dakkom', 'dikkom', 'dikkom', 'dakk', 'da'],
 ['dikkom', 'dakkom', 'dikkom', 'dikkom', 'dakk'],
 ['dig', 'det'],
 ['din', 'dag'],
 ['dikkom', 'dakkom', 'dikkom', 'dikkom', 'dakk'],
 ['dikkom', 'dakkom', 'dikkom', 'dikkom', 'dakk', 'din'],
 ['dikkom', 'dakkom', 'dikkom', 'dikkom', 'dakk'],
 ['traver', 'traar'],
 ['rennefok', 'rykemjell']]

In [21]:
from collections import Counter

# Tell antall ord
unique_words_in_alliterations = len(set([word for words in alliterations for word in words]))
print("Antall unike ord som inngår i bokstavrim i dette diktet: ", unique_words_in_alliterations)

# Finn ordrekka med flest bokstavrim på samme bokstav
top_count = max([len(words) for words in alliterations])

letters= Counter([word[0] for line in alliterations for word in line])

print("Største antall ord som inngår i samme bokstavrim: ", top_count)
print("Bokstaver som inngår i bokstavrimene: " )
letters

Antall unike ord som inngår i bokstavrim i dette diktet:  30
Største antall ord som inngår i samme bokstavrim:  6
Bokstaver som inngår i bokstavrimene: 


Counter({'d': 31,
         'h': 4,
         's': 4,
         'f': 2,
         'v': 2,
         'm': 2,
         'n': 2,
         'a': 2,
         't': 2,
         'r': 2})

### Finn bare bokstavrim som begynner på konsonanter

In [22]:
from poetry_analysis.alliteration import find_line_alliterations

# Vi gjenbruker eksempeldiktet vi lastet inn tidligere i variabelen "text"
alliterations = find_line_alliterations(text, letter_type="consonant")

alliterations

[['fjord', 'fjell'],
 ['heiopsan', 'her'],
 ['huld', 'helst'],
 ['vante', 'vis'],
 ['men', 'mange'],
 ['sakner', 'solens'],
 ['smaa', 'sig'],
 ['natt', 'naar'],
 ['dikkom', 'dakkom', 'dikkom', 'dikkom', 'dakk', 'da'],
 ['dikkom', 'dakkom', 'dikkom', 'dikkom', 'dakk'],
 ['dig', 'det'],
 ['din', 'dag'],
 ['dikkom', 'dakkom', 'dikkom', 'dikkom', 'dakk'],
 ['dikkom', 'dakkom', 'dikkom', 'dikkom', 'dakk', 'din'],
 ['dikkom', 'dakkom', 'dikkom', 'dikkom', 'dakk'],
 ['traver', 'traar'],
 ['rennefok', 'rykemjell']]

### Finn bare bokstavrim som begynner på vokaler

In [23]:
alliterations = find_line_alliterations(text, letter_type="vowel")

alliterations

[['av', 'alle']]

## Anafor

<!--Anafor: gjentakende ord, frase i starten av leddsetninger eller linjer -->

Anafor definerer vi som gjentakende ord eller fraser i begynnelsen av påfølgende verselinjer. 

Les mer om anafor: https://snl.no/anafor

Funksjonen `extract_anaphora` henter ut henholdsvis uni-, bi-, tri- og firegrammer fra påfølgende linjer

In [39]:
from poetry_analysis.anaphora import extract_anaphora

anaphora_ngrams = extract_anaphora(text)
anaphora_ngrams

{'1-grams': {'jeg': 3,
  'hun': 2,
  'dikkom': 5,
  'da': 2,
  'og': 4,
  'men': 2,
  'naar': 2,
  'min': 2},
 '2-grams': {'dikkom dakkom': 5},
 '3-grams': {'dikkom dakkom dikkom': 5},
 '4-grams': {'dikkom dakkom dikkom dikkom': 5}}

Funksjonen `extract_line_anaphora` henter ut fraser som gjentar seg på samme linje

In [25]:
from poetry_analysis.anaphora import extract_line_anaphora

line_anaphora = extract_line_anaphora(text)

line_anaphora

[{'line_id': 4, 'phrase': 'dikkom', 'count': 3},
 {'line_id': 13, 'phrase': 'dikkom', 'count': 3},
 {'line_id': 22, 'phrase': 'dikkom', 'count': 3},
 {'line_id': 31, 'phrase': 'dikkom', 'count': 3},
 {'line_id': 40, 'phrase': 'dikkom', 'count': 3}]

## Hvordan undersøke mange tekster samtidig?  

Vi kan jo lett se gjentakelsesmønstrene selv når vi ser på én tekst av gangen, men hva om du har tusenvis? 

Ta for eksempel korpuset [NORN Dikt](https://github.com/norn-uio/norn-poems) som har over 3000 korrekturleste dikt og som jeg har lastet inn i en jsonl-fil:  `norn_poems.jsonl`. 
Filen `metadata.jsonl` inneholder bibliografisk informasjon om bøkene som diktene er publisert i. 

In [28]:
import pandas as pd

# Last inn i en pandas dataramme
textdata = pd.read_json("norn_poems.jsonl", lines=True)
meta = pd.read_json("metadata.jsonl", lines=True)

df = meta.merge(textdata, on="poem_id")

### Enderim på mange tekster

Så kan vi bruke funksjonen `tag_text` på datarammen for å annotere enderimene: 

In [40]:
from poetry_analysis.rhyme_detection import tag_text

df["end_rhymes"] = df.text.apply(lambda t: list(tag_text(t)))

Resultatet er et nøstet hierarki av strofer og linjer og annotasjoner som hører til, så vi flater det ut med `pandas.json_normalize`: 

In [41]:
all_records = []

for _, row in df.iterrows():
    # Normaliser annotasjonsstrukturen for hver rad
    normalized = pd.json_normalize(
        row["end_rhymes"],
        record_path="verses",
        meta=["stanza_id", "rhyme_scheme"],
    )
    # Legg til dikt-ID så vi beholder koblingen til metadata
    normalized["poem_id"] = row["poem_id"]
    all_records.append(normalized)

# Slå sammen alle radene
expanded = pd.concat(all_records, ignore_index=True)

In [42]:
# Se på de fem første radene i datarammen med .head()
expanded.head()

,rhyme_score,rhyme_tag,text,transcription,tokens,syllables,last_token,rhymes_with,verse_id,stanza_id,rhyme_scheme,poem_id
0,0.0,a,"Ungdomsslægt, som fremad stevner",,"[ungdomsslægt, som, fremad, stevner]",None,stevner,NaN,0,0,ababcdcd,2641
1,0.0,b,"mod et sikkert mål,",,"[mod, et, sikkert, mål]",None,mål,NaN,1,0,ababcdcd,2641
2,1.0,a,"ungdomssind, som alting levner",,"[ungdomssind, som, alting, levner]",None,levner,0.0,2,0,ababcdcd,2641
3,1.0,b,fast som hærdet stål ;,,"[fast, som, hærdet, stål]",None,stål,1.0,3,0,ababcdcd,2641
4,0.0,c,ungdomsslægt med store tanker,,"[ungdomsslægt, med, store, tanker]",None,tanker,NaN,4,0,ababcdcd,2641


### Hvilke anaforer forekommer oftest?


In [ ]:

from poetry_analysis.anaphora import construct_anaphora_df

anaphora_df = construct_anaphora_df(expanded, anaphora_length=3)

In [48]:
anaphora_df.phrase.value_counts()

phrase
og det er         5
golgathamanden    5
kom og hjælp      5
seier skal vi     3
det er som        3
                 ..
husk på det       1
jeg synes at      1
for dig betød     1
saa det er        1
mon dette var     1
Name: count, Length: 340, dtype: int64

## Hvordan undersøke egne tekster?  
 

In [44]:
# Definer stien til en mappe og kjør funksjonene på hver av tekstene

In [46]:
dirpath=Path("eksempeldikt")

results = []
for file in dirpath.iterdir(): 
    rhymes = tag_poem_file(file)
    results.append(rhymes)

In [47]:
results

[[{'stanza_id': 0,
   'rhyme_scheme': 'abcd',
   'verses': [{'rhyme_score': 0,
     'rhyme_tag': 'a',
     'text': 'Et Aar idag!',
     'transcription': '',
     'tokens': ['et', 'aar', 'idag'],
     'syllables': None,
     'last_token': 'idag',
     'rhymes_with': None,
     'verse_id': 0},
    {'rhyme_score': 0,
     'rhyme_tag': 'b',
     'text': 'Du store miu,',
     'transcription': '',
     'tokens': ['du', 'store', 'miu'],
     'syllables': None,
     'last_token': 'miu',
     'rhymes_with': None,
     'verse_id': 1},
    {'rhyme_score': 0,
     'rhyme_tag': 'c',
     'text': 'saa tyk og rød',
     'transcription': '',
     'tokens': ['saa', 'tyk', 'og', 'rød'],
     'syllables': None,
     'last_token': 'rød',
     'rhymes_with': None,
     'verse_id': 2},
    {'rhyme_score': 0,
     'rhyme_tag': 'd',
     'text': 'og fed og fin!',
     'transcription': '',
     'tokens': ['og', 'fed', 'og', 'fin'],
     'syllables': None,
     'last_token': 'fin',
     'rhymes_with': None,
   